In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import os.path
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img

# **Training the model to recognize artist's style**

**Importing Wikiart dataset from Kaggle**

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"--provide the kaggle token here--"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download steubk/wikiart

In [ ]:
!unzip /content/wikiart.zip

In [10]:
rm /content/wikiart.zip

In [11]:
ds_path = "/content"
df_classes = pd.read_csv(f"{ds_path}/classes.csv")

**Preprocessing Function**



In [5]:
def resize(image, size):
    image = tf.image.resize(image, [size, size])
    return image


def preprocess_image(file_path, label, augmentation = True, rescale = True):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels=3)

    if augmentation:
        crop_p = tf.cast( tf.random.uniform([],0,1)<0.75, tf.float32)
        r= 1.0 + 0.5*np.random.random()*crop_p


        img = resize (img, size=int(IMG_SIZE*r))
        img = tf.image.random_crop(
            img, size=[IMG_SIZE, IMG_SIZE, 3]
        )


        img = tf.image.random_flip_left_right(
            img
        )

    else:
        img = resize (img, size=IMG_SIZE)

    if rescale:
        img = tf.cast( img, tf.float32) / 255.0

    return img, label

**Splitting the dataset into train, valid and test**

In [35]:
batch_size = 16
IMG_SIZE = 300


def create_ds(df_classes, ds_path):
    filenames = []
    labels = []
    for i in range(df_classes.shape[0]):
      x = df_classes.iloc[i]["filename"]
      if os.path.exists(f"{ds_path}/{x}"):   # If the image with the filename doesnot exist, then it won't be stored in the dataset
        filenames += [f"{ds_path}/{x}"]
        labels += [df_classes.iloc[i]["artist"]]
    ds = tf.data.Dataset.from_tensor_slices((filenames, labels))
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = configure_ds_for_performance(ds,batch_size, shuffle=True)
    return ds

def configure_ds_for_performance(ds, batch_size, shuffle, shuffle_buffer_size=32, shuffle_seed=42, reshuffle_each_iteration=True):
    if shuffle:
        ds = ds.shuffle(buffer_size=shuffle_buffer_size, seed=shuffle_seed, reshuffle_each_iteration = reshuffle_each_iteration)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds


le = LabelEncoder()
df_classes["artist"] = le.fit_transform(df_classes["artist"])   # Encoding the labels to integer values

X_train, X_temp, y_train, y_temp = train_test_split(df_classes, df_classes["artist"], test_size=0.3, random_state=42)   # 70% train, 30% validation and test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)   # 15% validation, 15% test datsets

train_ds = create_ds(X_train, ds_path)

valid_ds = create_ds(X_val, ds_path)

test_ds = create_ds(X_test, ds_path)

**Some samples of the dataset**

In [29]:
def show_samples (df_train, ds_path):

    df = df_train.sample(n=50, random_state=421)

    filenames = [ f"{ds_path}/{filename}" for filename in  df["filename"].values if os.path.exists(f"{ds_path}/{filename}")]
    labels = df["artist"]
    plt.figure(figsize=(30, 30))
    for n, (filename, label) in enumerate(zip(filenames,labels)):
        if n < 10*10:
            image, label = preprocess_image(filename, label, augmentation=False, rescale = False)

            ax = plt.subplot(10, 10, n+1)
            plt.imshow(image.numpy().astype("uint8"))
            ax.set_title( label )
            plt.axis("off")
    plt.show()

show_samples(X_train, ds_path)

**Some images after preprocessing**

In [ ]:
def show_augmentation (df_train, ds_path):

    df = df_train.sample(n=100, random_state=421)
    filenames = [ f"{ds_path}/{filename}" for filename in  df["filename"].values if os.path.exists(f"{ds_path}/{filename}")]
    labels = df["description"].values

    plt.figure(figsize=(15, 30))
    for n, (filename, label) in enumerate(zip(filenames,labels)):
        if n < 10:
            image, label = preprocess_image(filename, label, augmentation=False, rescale = False)
            ax = plt.subplot(10, 5, 5*n + 1)

            ax.imshow(image.numpy().astype("uint8"))
            ax.set_title( label )
            ax.axis("off")
            for i in range(4):
                ax = plt.subplot(10, 5, 5*n + 2+i)
                image_aug, label = preprocess_image(filename, label, augmentation=True, rescale = False)
                ax.imshow(image_aug.numpy().astype("uint8"))
                ax.axis("off")

show_augmentation(X_train, ds_path)

**Building a model using EfficientNetB3**

In [37]:
base_model = tf.keras.applications.EfficientNetB3(weights='imagenet')

50095040/50095040 [==============================] - 1s 0us/step


In [42]:
last_output = base_model.output

x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1119, activation="softmax")(x)   # In the dataset, number of artists = 1119

model = Model(base_model.input, x)

In [43]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


**Training the model using train and validation datasets**

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data = valid_ds)

**Testimg the model and visualising the performance**

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f'Test accuracy: {test_acc}')


plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()


# **Transfering the art style**

**Extracting outputs of intermediate layers of the model to extract style from the art**

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
style_layers = ['block1a_dwconv',
                'block1b_dwconv',
                'block2a_expand_conv',
                'block2b_expand_conv',
                'block2c_expand_conv',
                'block3a_expand_conv',
                'block3b_expand_conv',
                'block3c_expand_conv',
                'block4a_expand_conv',
                'block4b_expand_conv',
                'block4c_expand_conv',
                'block4d_expand_conv',
                'block4e_expand_conv',
                'block5a_expand_conv',
                'block5b_expand_conv',
                'block5c_expand_conv',
                'block5d_expand_conv',
                'block5e_expand_conv',
                'block6a_expand_conv',
                'block6b_expand_conv',
                'block6c_expand_conv',
                'block6d_expand_conv',
                'block6e_expand_conv',
                'block6f_expand_conv',
                'block7a_expand_conv',
                'block7b_expand_conv']

In [ ]:
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array /= 255.0
    return img_array

def build_model(style_path, content_shape=(256, 256, 3)):
    style_image = load_and_preprocess_image(style_path)

    model = Model.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=content_shape))

    style_outputs = [layer.output for layer in style_layers]


    style_model = Model([model.input], style_outputs)


    style_features = style_model(style_image)

    for layer in style_model.layers:
        layer.trainable = False

    model.add(layers.Lambda(lambda x: x * 0.8))
    model.add(layers.Lambda(lambda x: x + style_features))

    return model

**Functions to calculate Loss**

In [ ]:
def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

def content_loss(content, generated):
    return tf.reduce_mean(tf.square(content - generated))

def style_loss(style, generated):
    style_gram = gram_matrix(style)
    generated_gram = gram_matrix(generated)
    return tf.reduce_mean(tf.square(style_gram - generated_gram))

def total_variation_loss(image):
    x_deltas, y_deltas = image[:, 1:, :, :] - image[:, :-1, :, :], image[:, :, 1:, :] - image[:, :, :-1, :]
    return tf.reduce_mean(tf.square(x_deltas) + tf.square(y_deltas))

def total_loss(content, style, generated, content_weight=1e3, style_weight=1e-2, tv_weight=1e-2):
    content_loss_value = content_weight * content_loss(content, generated)
    style_loss_value = style_weight * style_loss(style, generated)
    tv_loss_value = tv_weight * total_variation_loss(generated)
    return content_loss_value + style_loss_value + tv_loss_value

**Transfering the style from style image to original image(content image)**

In [ ]:
def style_transfer(content_path, style_path, output_path, iterations=1000):

    content_image = load_and_preprocess_image(content_path)

    model = build_model(style_path, content_image.shape[1:])

    opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    for i in range(iterations):
        with tf.GradientTape() as tape:
            generated_image = model(content_image)
            loss = total_loss(content_image, style_image, generated_image)
        gradients = tape.gradient(loss, model.trainable_variables)
        opt.apply_gradients(zip(gradients, model.trainable_variables))

        if i % 100 == 0:
            print("Iteration {}: Loss: {}".format(i, loss))

    generated_image = tf.squeeze(generated_image, 0)
    generated_image = tf.clip_by_value(generated_image, 0, 1)
    generated_image = tf.image.convert_image_dtype(generated_image, dtype=tf.uint8)
    save_img(output_path, generated_image)

**An example for transfering the style and the visualisation of the result**

In [ ]:
style_path = '/content/abidin-dino_drawing-pain-1968.jpg'
content_path = '/content/download.jpg'
content_image = load_img(content_path)
style_image = load_img(style_path)
output_path = 'output_image.jpg'
style_transfer(content_path, style_path, output_path)

In [ ]:
plt.figure(figsize=(30,30))
plt.subplot(5,5,1)
plt.title("Base Image",fontsize=20)
img_original = load_img(content_path)
plt.imshow(img_original)

plt.subplot(5,5,1+1)
plt.title("Style Image",fontsize=20)
img_style = load_img(style_path)
plt.imshow(img_style)

plt.subplot(5,5,1+2)
plt.title("Final Image",fontsize=20)
imgx = load_img(output_path)
plt.imshow(imgx)

# **LIMITATIONS**

 

*   Testing the other EfficientNet models and selecting the best one might help make the model training better.
*   Using other metrics along with accuracy might make the model more efficient.
*   More preproccesing functions can be used on the art image to extract the style features more effectively.
*   Training the model on a subset of the Wikiart dataset will help running the code more faster since the dataset is too large.